In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import CTS as cts
from scipy.constants import *
from scipy.signal import hilbert
from scipy import ndimage
from sheffield_spetral import SheffieldThomson

sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis")
from data_analysis_utils import find_all_zerocrossing, hl_envelopes_idx

plt.rcParams.update({'font.size': 20})
%matplotlib qt

In [2]:
freq_in = [600e9] # Hz

ne = [1e12, 1e13, 1e13, 3e13] #[1e12, 5e12, 1e13, 5e13] #
Te = [10,3, 200, 500] #[5, 5, 5, 5] #
Ti = [3,1, 200, 500]#[1, 1, 1, 1] #


cl = ['r', 'g', 'b', 'y']
labels = ['LAPD condition 1', 'LAPD condition 2', 'TAE condition 1', 'TAE condition 2']

In [ ]:
lambda_in = c / freq_in[0] * 1e9

fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlabel(r"$\omega_s - \omega_i (rad/s)$")
ax.set_ylabel(r"$S(k, \omega)$")

for i in range(len(ne)):
    # Create calculator instance
    calculator = SheffieldThomson(
        probe_wavelength=lambda_in,  # nm
        T_e=Te[i],                   # eV
        T_i=Ti[i],                   # eV
        n_e=ne[i],                   # cm^-3
        scattering_angle=180,        # degrees
        ions="He+"                   # same as in notebook
    )
    

    alpha, omega_arr, omega_in, Skw = calculator.compute_spectral_density(delta_lam=5000, num_points=5000)
    ax.plot(omega_arr-omega_in, Skw, label=labels[i], color=cl[i])

    print(f"\nCondition {i+1}:")
    print(f"ne={ne[i]:.1e} cm^-3, Te={Te[i]} eV, Ti={Ti[i]} eV")
    print(f"Alpha parameter: {alpha:.3f}")
    integrated_Skw = calculator.integrate_spectral_density()
    print(f"Integrated spectral density: {integrated_Skw:.3e}")
    
    # You can also calculate the power ratio
    power = calculator.power_ratio(L=1)  # cm
    print(f"Power ratio (L=1cm): {power:.3e}")

plt.legend(title='$n_e(10^{13}cm^{-3})$')

In [21]:
freq_arr = np.linspace(300e9, 600e9, 1000)
lambda_in = c / freq_arr

plt.figure(figsize=(8, 6))

a_ls =cts.compute_alpha(lambda_in*100, 180, ne[0], Te[0])
plt.plot(freq_arr*1e-9, a_ls, label=f"LAPD condition 1", c='b')

a_ls =cts.compute_alpha(lambda_in*100, 180, ne[1], Te[1])
plt.plot(freq_arr*1e-9, a_ls, label=f"LAPD condition 2", c='r')

a_ls =cts.compute_alpha(lambda_in*100, 180, ne[2], Te[2])
plt.plot(freq_arr*1e-9, a_ls, '--',label=f"TAE condition 1", c='b')

a_ls =cts.compute_alpha(lambda_in*100, 180, ne[3], Te[3])
plt.plot(freq_arr*1e-9, a_ls, '--',label=f"TAE condition 2", c='r')

plt.xlabel(r"frequency (GHz)")
plt.ylabel(r"$\alpha=1/k\lambda_D$")
plt.yticks(fontweight='bold')
plt.legend(loc='upper right', fontsize=16)
plt.tight_layout()

In [18]:
freq_ls = np.arange(300e9, 600e9, 10e9)
plt.figure(figsize=(8, 5))
plt.xlabel("frequency (GHz)")
plt.ylabel("$P_s/P_iL (1/cm) $")

for i in range(len(ne)):
    pr_ls = []
    for freq in freq_ls:

        lambda_in = c / freq
        alpha, omega_arr, omega_in, Skw = cts.generate_spectral_density(lambda_in*1e9, Te[i], Ti[i], ne[i], delta_lam=5000, ions="H+")
        pr = cts.power_ratio(ne[i], omega_arr, Skw, scattering_angle=180, L=1)
        pr_ls.append(pr)

    plt.plot(freq_ls*1e-9, pr_ls, label=f"ne={ne[i]:.1e}, Te={Te[i]}, Ti={Ti[i]}")
plt.legend()
plt.tight_layout()

In [48]:
# Parameters
a = 5 # receiver radius in cm
f0 = 300e9 # units: Hz
omega_0 = 2*np.pi*f0
n_cycles = 32
dt = 0.05e-12 # units: second
sigma_t = 0.9e-12 # units: second
t0 = 5e-12 # units: second
n_e = 1e13  # cm^-3
B = 0.1 # Tesla

wpe = 5.64e4 * np.sqrt(n_e)
vg = c * np.sqrt(1 - wpe**2 / omega_0**2)

x_0 = 10 # cm
x_pl = 200 # cm
L_arr = np.arange(x_0, x_0+x_pl, 10) # units: meter

tarr, sig_in, freqs, signal_fft, center_ind = cts.generate_thz_waveform(f0/1e12, sigma_t*1e12, n_cycles, dt*1e12, t0*1e12, npd=100)
# tarr, sig_in, freqs, signal_fft = cts.generate_n_cycle_wave_packet(f0, sigma_t, n_cycles, dt, 0.1, 5000)
NT = len(tarr)
f_center = np.argwhere(freqs > 300e9)[0][0]
f_start = f_center - 1000
f_stop = f_center + 1000

t_start = center_ind - 2000
t_stop = center_ind + 2000

In [27]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(tarr[t_start:t_stop], sig_in[t_start:t_stop])
ax.set_xlabel('Time (ps)')
ax.set_ylabel('Amplitude')

plt.tight_layout()

plt.figure(figsize=(8, 6))
plt.plot(freqs[f_start:f_stop]/1e9, np.abs(signal_fft[f_start:f_stop]))

plt.yscale('log')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Amplitude')
plt.tight_layout()

signal_propagated, fft_propagated= cts.propagate_through_dispersive_medium(len(tarr), freqs, signal_fft, n_e, 1)

ax2 = ax.twinx()
ax2.plot(tarr-3680, signal_propagated, linewidth=2, label='Propagated Signal',c='orange',alpha=0.5)
ax2.set_yticks([])


plt.figure(figsize=(8, 6))
plt.plot(freqs[f_start:f_stop]/1e9, np.abs(signal_fft[f_start:f_stop]), linewidth=2, label='Propagated Signal')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Amplitude')
plt.yscale('log')
plt.tight_layout()
print(max(np.abs(signal_fft[f_start:f_stop])))

In [ ]:
l = (L_arr[0]-x_0)/1e2
signal_propagated, fft_propagated= cts.propagate_through_dispersive_medium(len(tarr), freqs, signal_fft, n_e, l)
t_center = (l/vg + n_cycles/(2*f0))*1e12
plt.figure(figsize=(8, 6))
plt.plot(tarr-t_center, signal_propagated)
plt.xlabel('Time (ps)')
plt.ylabel('Amplitude')


# Find the time window around t_center
time_window = 10  # ps
t_window_start = t_center - time_window
t_window_stop = t_center + time_window

# Find the indices corresponding to this time window
idx_window = np.where((tarr >= t_window_start) & (tarr <= t_window_stop))[0]
plt.plot(tarr[idx_window]-t_center, signal_propagated[idx_window])

# Find the peak-to-peak amplitude in this window
peak_to_peak = np.max(signal_propagated[idx_window]) - np.min(signal_propagated[idx_window])
print(f"Peak-to-peak amplitude in ±{time_window}ps window around t_center: {peak_to_peak}")

In [ ]:
lambda_in = c / f0 * 1e9

plt.figure(figsize=(8, 6))
plt.xlabel('Distance (m)')
plt.ylabel('$P_s/P_i$ (dB)')

for i in range(len(ne)):
    calculator = SheffieldThomson(lambda_in, Te[i], Ti[i], ne[i], scattering_angle=180, ions='H+')

    alpha, omega_arr, omega_in, Skw = calculator.compute_spectral_density(5000, num_points=f_stop-f_start)
    int_Skw = calculator.integrate_spectral_density()

    pr, pr_dB = calculator.power_ratio(L=3)

    pwarr = []
    for l in L_arr:
        solid_ang = 2*np.pi* (1 - 1 / np.sqrt(1+(a/(l))**2) )
        pwarr.append(10*np.log10(pr*solid_ang))

    plt.plot(L_arr/1e2, pwarr, label=labels[i])

plt.legend()

In [66]:
w0 = 2*np.pi*f0
omega_arr = np.repeat(w0, len(L_arr))
B_arr = np.repeat(B, len(L_arr))
ang_arr = cts.faraday_rotation_angle(omega_arr, n_e*1e6, B_arr, L_arr/1e2)

plt.figure(figsize=(8, 6))
plt.plot(L_arr/1e2, ang_arr, label='Forward')
plt.plot(L_arr/1e2, ang_arr*2, label='Return')
plt.legend()
plt.xlabel("Distance (m)")
plt.ylabel("Faraday rotation angle (deg)")
plt.tight_layout()